### Keras API Change

The *Adam* optimizer is updated in the Keras API.
it is now called *adam_v2*. Not exactly sure how this will affect the code.

### Tensorflow Keras Optimizer in the Tensorflow Module

has Adam as an optimizer.
to use it we need.

```python
from tensorflow.keras.optimizers import Adam
```


In [ ]:
import numpy as np
import tensorflow as tf
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential
from keras.models import load_model
import cv2
import datetime

In [ ]:
model_path = "../Models/v1_model.h5"
TRAIN_SET_PATH = "../gData/train"
TEST_SET_PATH = "../gData/test"

LEARNING_RATE = 1e-5
EPOCHS = 20
BATCH_SIZE = 16

In [ ]:
# BUILDING MODEL TO CLASSIFY BETWEEN MASK AND NO MASK
model = Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(
	optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE, decay=LEARNING_RATE / EPOCHS),
	loss="binary_crossentropy",
	metrics=["accuracy"]
)

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(
        TRAIN_SET_PATH,
        target_size=(150,150),
        batch_size=16 ,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        TEST_SET_PATH,
        target_size=(150,150),
        batch_size=16,
        class_mode='binary')

In [ ]:
# UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. 
# Please use `Model.fit`, which supports generators.

model_saved=model.fit_generator(
        training_set,
        epochs=10,
        validation_data=test_set,
        )

model.save(model_path, model_saved)

In [ ]:
model_saved_2 = model.fit(training_set, epochs=10, validation_data=test_set, verbose=2)

model.save("../Models/mask-detection-model_v2.h5", model_saved_2)

In [ ]:
#To test for individual images
model = load_model(model_path)


test_image_withMask = image.load_img(
    '../Data/test/with_mask/1-with-mask.jpg', 
    target_size=(150,150,3)
)

In [ ]:
test_image_withMask

In [ ]:
# converting the image to a numpy array
test_image_withMask = image.img_to_array(test_image_withMask)
test_image_withMask = np.expand_dims(test_image_withMask,axis=0)
model.predict(test_image_withMask)[0][0]

In [ ]:
# IMPLEMENTING LIVE DETECTION OF FACE MASK

mymodel = load_model(model_path)

cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture(1)
face_cascade = cv2.CascadeClassifier('../Classifiers/cascade_frontalface_default.xml')

while cap.isOpened():
    _, img=cap.read()
    face = face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=8)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict(test_image)[0][0]

        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,
                'NO MASK',
                ((x+w)//2,y+h+20),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0,0,255),
                3
            )
            print("NO MASK")
        
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,
                'MASK',
                ((x+w)//2,y+h+20),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0,255,0),
                3
            )
            print("MASK")

        datet=str(datetime.datetime.now())
        cv2.putText(img,
            datet,
            (400,450),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255,255,255),
            1
        )

    cv2.imshow('Viewport',img)

    if cv2.waitKey(1) == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()